In [86]:
import json
import csv
#import tweepy
import re
import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd
#import GetOldTweets3 as got
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
import numpy as np
from nltk import tokenize
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [87]:
df=pd.read_csv('../../Datasets/Quora/Data/pesu_quora_qa.csv')
df_details=pd.read_csv('../../Datasets/Quora/Data/pesu_quora_details.csv')

In [88]:
df

,question,answers
0,Are vehicles allowed in PES university hostel?...,"Yes, Vehicles are allowed in the college hostel"
1,Why has PESU scored such a low ranking in the ...,The reason their ranking has dropped is that t...
2,Why is the highest package offered at NIT Sura...,I don’t know about the highest package in RVCE...
3,Why is the highest package offered at NIT Sura...,"Nope, this is not true. You can say that the h..."
4,Why is the highest package offered at NIT Sura...,Because the placement stats published by any I...
...,...,...
161,Why is PES University placement cell not picki...,All colleges are doing their best to convince ...
162,What is PES University doing for students who ...,What can the college do? They will have convin...
163,Is the PESSAT exam applicable for diploma stud...,No. The PESSAT exam is only for PU( class -12t...
164,Is the PESSAT exam applicable for diploma stud...,No the PESSAT exam is only applicable to Class...


In [89]:
df_details

,Topic,Title,Followers,Answers,Ratio,Engagement potential,Last action,Parsed time,Question link
0,PES-University-1,"Which is the best college for a BBA, IFIM or PES?",1,0,1/0,No,Followed,2020-06-18,https://www.quora.com/unanswered/Which-is-the-...
1,PES-University-1,Are vehicles allowed in PES university hostel?...,1,1,1/1,No,Followed,2020-06-17,https://www.quora.com/Are-vehicles-allowed-in-...
2,PES-University-1,What is PESUs opinion on not cancelling exams?...,2,0,2/0,No,Followed,2020-06-14,https://www.quora.com/unanswered/What-is-PESUs...
3,PES-University-1,Why has PESU scored such a low ranking in the ...,1,1,1/1,No,Followed,2020-06-14,https://www.quora.com/Why-has-PESU-scored-such...
4,PES-University-1,Why is the highest package offered at NIT Sura...,2,3,2/3,No,Followed,2020-06-11,https://www.quora.com/Why-is-the-highest-packa...
...,...,...,...,...,...,...,...,...,...
1095,PES-University-1,Have the classes started for the MBA at PES Un...,0,0,0/0,No,Requested,2018-10-03,https://www.quora.comhttps://www.quora.com/una...
1096,PES-University-1,How is the PESIT South Campus for an MTech?,2,0,2/0,No,Followed,2018-10-03,https://www.quora.comhttps://www.quora.com/una...
1097,PES-University-1,Is it possible to join BTech in ECE at PES Uni...,1,2,1/2,No,Requested,2018-10-02,https://www.quora.comhttps://www.quora.com/Is-...
1098,PES-University-1,Is there a way to deal with the handicap when ...,1,0,1/0,No,Followed,2018-09-30,https://www.quora.comhttps://www.quora.com/una...


In [90]:
df=df.reset_index()
df_details=df_details.reset_index()

In [91]:
df_details.rename(columns = {'Title':'question'}, inplace = True) 

In [92]:
df_merge = pd.merge(df_details, df, on=['question'], how='inner')
df_merge

,index_x,Topic,question,Followers,Answers,Ratio,Engagement potential,Last action,Parsed time,Question link,index_y,answers
0,0,PES-University-1,"Which is the best college for a BBA, IFIM or PES?",1,0,1/0,No,Followed,2020-06-18,https://www.quora.com/unanswered/Which-is-the-...,42,IFIM - Bangalore is better than PES University...
1,0,PES-University-1,"Which is the best college for a BBA, IFIM or PES?",1,0,1/0,No,Followed,2020-06-18,https://www.quora.com/unanswered/Which-is-the-...,125,IFIM - Bangalore is better than PES University...
2,21,PES-University-1,"Which is the best college for a BBA, IFIM or PES?",1,1,1/1,No,Followed,2020-06-18,https://www.quora.com/Which-is-the-best-colleg...,42,IFIM - Bangalore is better than PES University...
3,21,PES-University-1,"Which is the best college for a BBA, IFIM or PES?",1,1,1/1,No,Followed,2020-06-18,https://www.quora.com/Which-is-the-best-colleg...,125,IFIM - Bangalore is better than PES University...
4,1,PES-University-1,Are vehicles allowed in PES university hostel?...,1,1,1/1,No,Followed,2020-06-17,https://www.quora.com/Are-vehicles-allowed-in-...,0,"Yes, Vehicles are allowed in the college hostel"
...,...,...,...,...,...,...,...,...,...,...,...,...
487,19,PES-University-1,Which college is best among PES and Jain Unive...,1,2,1/2,No,Followed,2020-05-29,https://www.quora.com/Which-college-is-best-am...,124,If you are looking for placements prefer Jain ...
488,58,PES-University-1,Which college is best among PES and Jain Unive...,1,2,1/2,No,Followed,2020-05-29,https://www.quora.comhttps://www.quora.com/Whi...,40,Jain university is better. Jain has a good cos...
489,58,PES-University-1,Which college is best among PES and Jain Unive...,1,2,1/2,No,Followed,2020-05-29,https://www.quora.comhttps://www.quora.com/Whi...,41,If you are looking for placements prefer Jain ...
490,58,PES-University-1,Which college is best among PES and Jain Unive...,1,2,1/2,No,Followed,2020-05-29,https://www.quora.comhttps://www.quora.com/Whi...,123,Jain university is better. Jain has a good cos...


In [93]:
df_merge=df_merge.drop(['index_x','index_y'],axis=1)
df_merge

,Topic,question,Followers,Answers,Ratio,Engagement potential,Last action,Parsed time,Question link,answers
0,PES-University-1,"Which is the best college for a BBA, IFIM or PES?",1,0,1/0,No,Followed,2020-06-18,https://www.quora.com/unanswered/Which-is-the-...,IFIM - Bangalore is better than PES University...
1,PES-University-1,"Which is the best college for a BBA, IFIM or PES?",1,0,1/0,No,Followed,2020-06-18,https://www.quora.com/unanswered/Which-is-the-...,IFIM - Bangalore is better than PES University...
2,PES-University-1,"Which is the best college for a BBA, IFIM or PES?",1,1,1/1,No,Followed,2020-06-18,https://www.quora.com/Which-is-the-best-colleg...,IFIM - Bangalore is better than PES University...
3,PES-University-1,"Which is the best college for a BBA, IFIM or PES?",1,1,1/1,No,Followed,2020-06-18,https://www.quora.com/Which-is-the-best-colleg...,IFIM - Bangalore is better than PES University...
4,PES-University-1,Are vehicles allowed in PES university hostel?...,1,1,1/1,No,Followed,2020-06-17,https://www.quora.com/Are-vehicles-allowed-in-...,"Yes, Vehicles are allowed in the college hostel"
...,...,...,...,...,...,...,...,...,...,...
487,PES-University-1,Which college is best among PES and Jain Unive...,1,2,1/2,No,Followed,2020-05-29,https://www.quora.com/Which-college-is-best-am...,If you are looking for placements prefer Jain ...
488,PES-University-1,Which college is best among PES and Jain Unive...,1,2,1/2,No,Followed,2020-05-29,https://www.quora.comhttps://www.quora.com/Whi...,Jain university is better. Jain has a good cos...
489,PES-University-1,Which college is best among PES and Jain Unive...,1,2,1/2,No,Followed,2020-05-29,https://www.quora.comhttps://www.quora.com/Whi...,If you are looking for placements prefer Jain ...
490,PES-University-1,Which college is best among PES and Jain Unive...,1,2,1/2,No,Followed,2020-05-29,https://www.quora.comhttps://www.quora.com/Whi...,Jain university is better. Jain has a good cos...


In [94]:
df_merge=df_merge.drop_duplicates()

In [95]:
df_merge=df_merge.reset_index()

In [96]:
df_merge=df_merge.drop(['index'],axis=1)

In [97]:
def vader_score(paragraph):
    analyzer = SentimentIntensityAnalyzer()
    paragraphSentiments = 0.0
    avg = -1
    sentence_list = tokenize.sent_tokenize(paragraph)
    size = len(sentence_list)
    #print(len((sentence_list))
    for sentence in sentence_list :
        vs = analyzer.polarity_scores(sentence)
        paragraphSentiments += vs["compound"]
    if( size == 0):
        print("zero")
    else:
        avg = (paragraphSentiments / size)
    return avg
df_merge['compound'] = df_merge['answers'].apply(lambda x: vader_score(x))
i = df_merge[df_merge['compound'] == -1].index
df_merge.drop(i)

,Topic,question,Followers,Answers,Ratio,Engagement potential,Last action,Parsed time,Question link,answers,compound
0,PES-University-1,"Which is the best college for a BBA, IFIM or PES?",1,0,1/0,No,Followed,2020-06-18,https://www.quora.com/unanswered/Which-is-the-...,IFIM - Bangalore is better than PES University...,0.293600
1,PES-University-1,"Which is the best college for a BBA, IFIM or PES?",1,1,1/1,No,Followed,2020-06-18,https://www.quora.com/Which-is-the-best-colleg...,IFIM - Bangalore is better than PES University...,0.293600
2,PES-University-1,Are vehicles allowed in PES university hostel?...,1,1,1/1,No,Followed,2020-06-17,https://www.quora.com/Are-vehicles-allowed-in-...,"Yes, Vehicles are allowed in the college hostel",0.401900
3,PES-University-1,Are vehicles allowed in PES university hostel?...,1,1,1/1,No,Followed,2020-06-17,https://www.quora.comhttps://www.quora.com/Are...,"Yes, Vehicles are allowed in the college hostel",0.401900
4,PES-University-1,Why has PESU scored such a low ranking in the ...,1,1,1/1,No,Followed,2020-06-14,https://www.quora.com/Why-has-PESU-scored-such...,The reason their ranking has dropped is that t...,0.106067
...,...,...,...,...,...,...,...,...,...,...,...
79,PES-University-1,I am planning to do my M.Tech in ECE from PES ...,3,1,3/1,No,Followed,2020-05-30,https://www.quora.comhttps://www.quora.com/I-a...,Getting placement in core branches is hard. Sa...,0.105325
80,PES-University-1,Which college is best among PES and Jain Unive...,1,2,1/2,No,Followed,2020-05-29,https://www.quora.com/Which-college-is-best-am...,Jain university is better. Jain has a good cos...,0.440400
81,PES-University-1,Which college is best among PES and Jain Unive...,1,2,1/2,No,Followed,2020-05-29,https://www.quora.com/Which-college-is-best-am...,If you are looking for placements prefer Jain ...,0.510600
82,PES-University-1,Which college is best among PES and Jain Unive...,1,2,1/2,No,Followed,2020-05-29,https://www.quora.comhttps://www.quora.com/Whi...,Jain university is better. Jain has a good cos...,0.440400


In [98]:
#classifying sentiments as positive,negative and neutral
def sentiment(compound):
    res = ""
    if(compound >= 0.05):
        res="Positive"
    elif(compound <= - 0.05):
        res="Negative"
    else:
        res="Neutral"
    return res

df_merge['Sentiment'] = df_merge['compound'].apply(lambda x: sentiment(x))

In [99]:
df_merge

,Topic,question,Followers,Answers,Ratio,Engagement potential,Last action,Parsed time,Question link,answers,compound,Sentiment
0,PES-University-1,"Which is the best college for a BBA, IFIM or PES?",1,0,1/0,No,Followed,2020-06-18,https://www.quora.com/unanswered/Which-is-the-...,IFIM - Bangalore is better than PES University...,0.293600,Positive
1,PES-University-1,"Which is the best college for a BBA, IFIM or PES?",1,1,1/1,No,Followed,2020-06-18,https://www.quora.com/Which-is-the-best-colleg...,IFIM - Bangalore is better than PES University...,0.293600,Positive
2,PES-University-1,Are vehicles allowed in PES university hostel?...,1,1,1/1,No,Followed,2020-06-17,https://www.quora.com/Are-vehicles-allowed-in-...,"Yes, Vehicles are allowed in the college hostel",0.401900,Positive
3,PES-University-1,Are vehicles allowed in PES university hostel?...,1,1,1/1,No,Followed,2020-06-17,https://www.quora.comhttps://www.quora.com/Are...,"Yes, Vehicles are allowed in the college hostel",0.401900,Positive
4,PES-University-1,Why has PESU scored such a low ranking in the ...,1,1,1/1,No,Followed,2020-06-14,https://www.quora.com/Why-has-PESU-scored-such...,The reason their ranking has dropped is that t...,0.106067,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...
79,PES-University-1,I am planning to do my M.Tech in ECE from PES ...,3,1,3/1,No,Followed,2020-05-30,https://www.quora.comhttps://www.quora.com/I-a...,Getting placement in core branches is hard. Sa...,0.105325,Positive
80,PES-University-1,Which college is best among PES and Jain Unive...,1,2,1/2,No,Followed,2020-05-29,https://www.quora.com/Which-college-is-best-am...,Jain university is better. Jain has a good cos...,0.440400,Positive
81,PES-University-1,Which college is best among PES and Jain Unive...,1,2,1/2,No,Followed,2020-05-29,https://www.quora.com/Which-college-is-best-am...,If you are looking for placements prefer Jain ...,0.510600,Positive
82,PES-University-1,Which college is best among PES and Jain Unive...,1,2,1/2,No,Followed,2020-05-29,https://www.quora.comhttps://www.quora.com/Whi...,Jain university is better. Jain has a good cos...,0.440400,Positive


In [100]:
def convert(score):
    OldRange = 2  
    NewRange = 4  
    NewValue = (((score + 1) * NewRange) / OldRange) + 1
    return NewValue
df_merge['score'] = df_merge['compound'].apply(lambda x: convert(x))
df_merge
def assignpred(n):
    if n>1 and n<3:
        return max((n-1),1)
    if n>3 and n<5:
        return min((n+1),5)

In [101]:
def normalize(feature):
    max_value = df_merge[feature].max()
    min_value = df_merge[feature].min()
    result = (df_merge[feature] - min_value) / (max_value - min_value)
    return result

In [102]:
df_merge['followers_n']=normalize('Followers')

In [103]:
df_merge

,Topic,question,Followers,Answers,Ratio,Engagement potential,Last action,Parsed time,Question link,answers,compound,Sentiment,score,followers_n
0,PES-University-1,"Which is the best college for a BBA, IFIM or PES?",1,0,1/0,No,Followed,2020-06-18,https://www.quora.com/unanswered/Which-is-the-...,IFIM - Bangalore is better than PES University...,0.293600,Positive,3.587200,0.000000
1,PES-University-1,"Which is the best college for a BBA, IFIM or PES?",1,1,1/1,No,Followed,2020-06-18,https://www.quora.com/Which-is-the-best-colleg...,IFIM - Bangalore is better than PES University...,0.293600,Positive,3.587200,0.000000
2,PES-University-1,Are vehicles allowed in PES university hostel?...,1,1,1/1,No,Followed,2020-06-17,https://www.quora.com/Are-vehicles-allowed-in-...,"Yes, Vehicles are allowed in the college hostel",0.401900,Positive,3.803800,0.000000
3,PES-University-1,Are vehicles allowed in PES university hostel?...,1,1,1/1,No,Followed,2020-06-17,https://www.quora.comhttps://www.quora.com/Are...,"Yes, Vehicles are allowed in the college hostel",0.401900,Positive,3.803800,0.000000
4,PES-University-1,Why has PESU scored such a low ranking in the ...,1,1,1/1,No,Followed,2020-06-14,https://www.quora.com/Why-has-PESU-scored-such...,The reason their ranking has dropped is that t...,0.106067,Positive,3.212133,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,PES-University-1,I am planning to do my M.Tech in ECE from PES ...,3,1,3/1,No,Followed,2020-05-30,https://www.quora.comhttps://www.quora.com/I-a...,Getting placement in core branches is hard. Sa...,0.105325,Positive,3.210650,0.166667
80,PES-University-1,Which college is best among PES and Jain Unive...,1,2,1/2,No,Followed,2020-05-29,https://www.quora.com/Which-college-is-best-am...,Jain university is better. Jain has a good cos...,0.440400,Positive,3.880800,0.000000
81,PES-University-1,Which college is best among PES and Jain Unive...,1,2,1/2,No,Followed,2020-05-29,https://www.quora.com/Which-college-is-best-am...,If you are looking for placements prefer Jain ...,0.510600,Positive,4.021200,0.000000
82,PES-University-1,Which college is best among PES and Jain Unive...,1,2,1/2,No,Followed,2020-05-29,https://www.quora.comhttps://www.quora.com/Whi...,Jain university is better. Jain has a good cos...,0.440400,Positive,3.880800,0.000000


In [104]:
engagement_median = df_merge['followers_n'].median()
df_merge["engagement"] = df_merge['followers_n'].apply(lambda x: 1 if x >= engagement_median else 0)

In [105]:
df_merge['sentiment_score']=df_merge['score']
df_eng1=df_merge[df_merge.engagement==1]
#if engagement score is greater than the median, increase the positive and negative sentiment
df_eng1['sentiment_score']=df_eng1['score'].apply(lambda x: assignpred(x))
df_merge[df_merge.engagement==1]=df_eng1
df_merge

C:\Users\Kirthika\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Topic,question,Followers,Answers,Ratio,Engagement potential,Last action,Parsed time,Question link,answers,compound,Sentiment,score,followers_n,engagement,sentiment_score
0,PES-University-1,"Which is the best college for a BBA, IFIM or PES?",1,0,1/0,No,Followed,2020-06-18,https://www.quora.com/unanswered/Which-is-the-...,IFIM - Bangalore is better than PES University...,0.293600,Positive,3.587200,0.000000,0,3.587200
1,PES-University-1,"Which is the best college for a BBA, IFIM or PES?",1,1,1/1,No,Followed,2020-06-18,https://www.quora.com/Which-is-the-best-colleg...,IFIM - Bangalore is better than PES University...,0.293600,Positive,3.587200,0.000000,0,3.587200
2,PES-University-1,Are vehicles allowed in PES university hostel?...,1,1,1/1,No,Followed,2020-06-17,https://www.quora.com/Are-vehicles-allowed-in-...,"Yes, Vehicles are allowed in the college hostel",0.401900,Positive,3.803800,0.000000,0,3.803800
3,PES-University-1,Are vehicles allowed in PES university hostel?...,1,1,1/1,No,Followed,2020-06-17,https://www.quora.comhttps://www.quora.com/Are...,"Yes, Vehicles are allowed in the college hostel",0.401900,Positive,3.803800,0.000000,0,3.803800
4,PES-University-1,Why has PESU scored such a low ranking in the ...,1,1,1/1,No,Followed,2020-06-14,https://www.quora.com/Why-has-PESU-scored-such...,The reason their ranking has dropped is that t...,0.106067,Positive,3.212133,0.000000,0,3.212133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,PES-University-1,I am planning to do my M.Tech in ECE from PES ...,3,1,3/1,No,Followed,2020-05-30,https://www.quora.comhttps://www.quora.com/I-a...,Getting placement in core branches is hard. Sa...,0.105325,Positive,3.210650,0.166667,0,3.210650
80,PES-University-1,Which college is best among PES and Jain Unive...,1,2,1/2,No,Followed,2020-05-29,https://www.quora.com/Which-college-is-best-am...,Jain university is better. Jain has a good cos...,0.440400,Positive,3.880800,0.000000,0,3.880800
81,PES-University-1,Which college is best among PES and Jain Unive...,1,2,1/2,No,Followed,2020-05-29,https://www.quora.com/Which-college-is-best-am...,If you are looking for placements prefer Jain ...,0.510600,Positive,4.021200,0.000000,0,4.021200
82,PES-University-1,Which college is best among PES and Jain Unive...,1,2,1/2,No,Followed,2020-05-29,https://www.quora.comhttps://www.quora.com/Whi...,Jain university is better. Jain has a good cos...,0.440400,Positive,3.880800,0.000000,0,3.880800


In [106]:
df_merge.rename(columns = {'score':'old_score'}, inplace = True) 

In [107]:
df_merge.to_csv('../../Datasets/Quora/Data/PesuQuoraWSA.csv')